# Deep Learning

* This week, we'll learn about neural nets and build a model for classifying images of clothes
* We will use Tensorflow

## Data: Fashion Classification

* https://www.kaggle.com/datasets/zalando-research/fashionmnist